### Connect to IB:

In [1]:
from pprint import *
from ib_insync import *
util.startLoop()  # uncomment this line when in a notebook

ib = IB() # making a IB class
ib.connect('127.0.0.1', 7497, clientId=2)
# ib.connect('10.0.0.209', 7497, clientId=1)

<IB connected to 127.0.0.1:7497 clientId=2>

Error 201, reqId 84: Order rejected - reason:YOUR ORDER IS NOT ACCEPTED. IN ORDER TO OBTAIN THE DESIRED POSITION YOUR NET LIQ [1002709.80 USD] MUST EXCEED THE MARGIN REQ [4802859.60 USD]
Canceled order: Trade(contract=Future(conId=620731036, symbol='MES', lastTradeDateOrContractMonth='20240621', multiplier='5', exchange='CME', currency='USD', localSymbol='MESM4', tradingClass='MES'), order=LimitOrder(orderId=84, clientId=2, permId=1758116108, action='SELL', totalQuantity=7.0, lmtPrice=5307.25, auxPrice=0.0, tif='GTC'), orderStatus=OrderStatus(orderId=84, status='Cancelled', filled=0.0, remaining=7.0, avgFillPrice=0.0, permId=1758116108, parentId=0, lastFillPrice=0.0, clientId=2, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 4, 1, 15, 10, 22, 587141, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 4, 1, 15, 10, 22, 791028, tzinfo=datetime.timezone.utc), status='Submitted

### Verify contract, here use Future

In [2]:
f = Future(symbol='MES', lastTradeDateOrContractMonth='202406', exchange='CME', currency='USD') # valid instrument - Future
contract = ib.qualifyContracts(f)[0]
pprint(contract)

Future(conId=620731036, symbol='MES', lastTradeDateOrContractMonth='20240621', multiplier='5', exchange='CME', currency='USD', localSymbol='MESM4', tradingClass='MES')


1st, a buy market order for X (default = 10) shares, return the trade.
in the function, 3 values are used for print: 
- mkt_buy_whether_filled, 
- mkt_buy_quantity, 
- mkt_buy_filled_price, 

2nd, a stop loss sell order (STP), use two of the 1st trade's returned values as input, but set the stop price as - 10
- shares = mkt_buy_quantity
- price = mkt_buy_filled_price - 10

3rd, a profit limit sell order (LMT), use the same two of the 1st trade's returned values as input, but set the shares as - 2, profit lmt price as + 10
- shares = mkt_buy_quantity -2
- price = mkt_buy_filled_price + 10

In [16]:
def mkt_buy(shares = 10):
    mkt_buy_ord = MarketOrder('BUY', totalQuantity=shares)
    mkt_buy_trades = ib.placeOrder(contract, mkt_buy_ord)
    ib.sleep(2)
    # pprint(mkt_buy_trades) #等2秒过后，这个时候的order已经被fill了，所以打印出来的信息，就已经包括了所有的order的信息了
    print('Market Buy Order: \n\t {0} \n\t for {1} shares \n\t @ {2}'.format(mkt_buy_trades.orderStatus.status, mkt_buy_trades.fills[0].execution.shares, mkt_buy_trades.fills[0].execution.price))
    return(mkt_buy_trades)


def stp_sell(shares, price):
    stp_ord = StopOrder(action='SELL', totalQuantity=shares, stopPrice=price, tif='GTC', outsideRth=True)
    stp_trades = ib.placeOrder(contract, stp_ord)
    # pprint(stp_trades)
    print('Stop Loss Order: \n\t {0} \n\t for {1} shares\n\t @ {2}'.format(stp_trades.orderStatus.status, stp_trades.order.totalQuantity, stp_trades.order.auxPrice))
    return(stp_trades)


def lmt_sell(shares, price):
    lmt_ord = LimitOrder(action='SELL', totalQuantity=shares, lmtPrice=price, tif='GTC')
    lmt_trades = ib.placeOrder(contract, lmt_ord)
    # pprint(lmt_trades)
    print('Profit Order: \n\t {0} \n\t for {1} shares\n\t @ {2}'.format(lmt_trades.orderStatus.status, lmt_trades.order.totalQuantity, lmt_trades.order.lmtPrice))
    return(lmt_trades)


#1, market price buy order 
put_mkt_buy_quantity = int(input('How many shares to buy? ') or '10')
put_mkt_buy_trade = mkt_buy(put_mkt_buy_quantity)

put_mkt_buy_trade_filled_price = put_mkt_buy_trade.fills[0].execution.price
put_mkt_buy_trade_filled_quantity = put_mkt_buy_trade.fills[0].execution.shares

#2, stop loss STP order
put_stp_sell_trade = stp_sell(put_mkt_buy_trade_filled_quantity, put_mkt_buy_trade_filled_price - 10)


#3, profit LMT order
put_lmt_sell_trade = lmt_sell(put_mkt_buy_trade_filled_quantity - 2, put_mkt_buy_trade_filled_price + 10)
ib.sleep(1)
print(put_lmt_sell_trade.orderStatus.status)



Market Buy Order: 
	 Filled 
	 for 10.0 shares 
	 @ 5285.5
Stop Loss Order: 
	 PendingSubmit 
	 for 10.0 shares
	 @ 5275.5
Profit Order: 
	 PendingSubmit 
	 for 8.0 shares
	 @ 5295.5
Submitted


<span style="color:red; font-size: 30px">
adjust the stop loss order STP to proper price
</span>


In [17]:
assert put_stp_sell_trade in ib.openTrades()
assert put_stp_sell_trade.orderStatus.status == 'PreSubmitted'

print('Current Stop Loss Order Price: ' + str(put_stp_sell_trade.order.auxPrice))
put_stp_sell_trade.order.auxPrice = float(input('What is the proper stop price? ')or put_stp_sell_trade.order.auxPrice)
adj_stp_sell_trade = ib.placeOrder(contract, put_stp_sell_trade.order)
print('Stop Order Price now changed to '+ str(put_stp_sell_trade.order.auxPrice))



Current Stop Loss Order Price: 5275.5
Stop Order Price now changed to 5283.0


<span style="color:blue; font-size: 30px">
adjust the profit taken order LMT to proper shares numbers(if needed) and price
</span>

In [19]:
assert put_lmt_sell_trade in ib.openTrades()
assert put_lmt_sell_trade.orderStatus.status == 'Submitted'
print('Current Profit Taken Order Quantiny: ' + str(put_lmt_sell_trade.order.totalQuantity))
put_lmt_sell_trade.order.totalQuantity = float(input('Quantity number change to? (hit enter for no change)') or put_lmt_sell_trade.order.totalQuantity)

print('Current Profit Taken Order Price: ' + str(put_lmt_sell_trade.order.lmtPrice))
put_lmt_sell_trade.order.lmtPrice = float(input('What is the proper LMT price? ') or put_lmt_sell_trade.order.lmtPrice)
adj_lmt_sell_trade = ib.placeOrder(contract, put_lmt_sell_trade.order)
print('Profit Taken Order LTM Quantity now changed to: ' + str(put_lmt_sell_trade.order.totalQuantity))
print('Profit Taken Order LTM Price now changed to: ' + str(put_lmt_sell_trade.order.lmtPrice))

Current Profit Taken Order Quantiny: 8.0
Current Profit Taken Order Price: 5295.5
Profit Taken Order LTM Quantity now changed to: 7.0
Profit Taken Order LTM Price now changed to: 5287.0


In [21]:
##### 很遗憾，not working as expected

while (adj_lmt_sell_trade.orderStatus.status == 'Submitted' and adj_stp_sell_trade.orderStatus.status == 'PreSubmitted'):
    continue
else: 
    if adj_lmt_sell_trade.orderStatus.status == 'Filled':
        print('Profit Taken Order Filled: \n\t sell {0} shares @ {1}'.format(adj_lmt_sell_trade.order.totalQuantity, adj_lmt_sell_trade.fills[0].execution.price))
        adj_stp_sell_trade.order.totalQuantity -= adj_lmt_sell_trade.fills[0].execution.shares
        adj_stp_sell_trade.order.lmtPrice = put_mkt_buy_trade_filled_price - 2
        after_lmt_stp_sell_trade = ib.placeOrder(contract, adj_stp_sell_trade.order)
        print('Profit Taken Order Filled, now STP changed to {0} shares @ {1}'.format(after_lmt_stp_sell_trade.order.totalQuantity, after_lmt_stp_sell_trade.order.auxPrice))
    elif adj_stp_sell_trade.orderStatus.status == 'Filled':
        print('Stop Loss Order Filled: \n\t sell all {0} shares at stop loss price @ {1}'.format(adj_stp_sell_trade.order.totalQuantity, adj_stp_sell_trade.fills[0].execution.price))
        after_stp_lmt_sell_trade = ib.cancelOrder(adj_lmt_sell_trade.order)
        print('Stop Loss Order Filled, cancel Profit Taken Order')




Profit Taken Order Filled: 
	 sell 7.0 shares @ 5287.0
Profit Taken Order Filled, now STP changed to 3.0 shares @ 5283.0
